In [ ]:
!pip install google-cloud-aiplatform
!pip install --user datasets
!pip install --user google-cloud-pipeline-components

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.1/269.1 kB 18.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9

In [ ]:
import IPython
from google.cloud import aiplatform
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
import vertexai
PROJECT_ID= "extreme-ratio-466001-h3"
vertexai.init(project=PROJECT_ID)

In [ ]:
project_id = "extreme-ratio-466001-h3"
region = "us-central1"
REGION = "us-central1"

In [ ]:
! gcloud config set project {project_id}

Updated property [core/project].


In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
import vertexai
vertexai.init(project=PROJECT_ID, location=REGION)
import kfp
import sys
import uuid
import json
import vertexai
import pandas as pd
from google.auth import default
from datasets import load_dataset
from google.cloud import aiplatform
from vertexai.preview.language_models import TextGenerationModel, EvaluationTextSummarizationSpec

In [ ]:
df = pd.read_json("/content/train.jsonl", lines=True)
df.head()

,question,answer,references
0,in football whats the point of wasting the fir...,The point of wasting the first two plays with ...,"[As you can see from these highlights, most of..."
1,Why are different tiers (regular < mid < premi...,The 10 cent difference between the different t...,[ELI5: Why are different tiers (regular<mid<pr...
2,Stars and Visibility,The visibility of stars from a specific locati...,"[Today, travelers trek to the ends of the Eart..."
3,How do we know all the money the government is...,It is difficult to know for sure that the mone...,"[Where there is no arrangement, your bank or b..."
4,What are good and bad sides of manual and auto...,The good sides of manual transmission are that...,[They’ve made it look like the best of both wo...


In [ ]:
print(df.shape)

(43579, 3)


In [ ]:
df = df.drop('references', axis=1)

In [ ]:
print(df.shape)

(43579, 2)


In [ ]:
model_display_name = "fintuning-model"
tuned_model = TextGenerationModel.from_pretrained("text-bison@002")
tuned_model.tune_model(
    training_data=df,
    train_steps = 100,
    tuning_job_location="europe-west4",
    tuned_model_location="europe-west4"
)

In [ ]:
response = tuned_model.predict("What are the metrics on which deepseek was evaluated")
print(response.text)

In [ ]:
tuned_model_name = tuned_model._endpoint.gca_resource.deployed_models[0].model
tuned_model_1 = TextGenerationModel.get_tuned_model(tuned_model_name)
response = tuned_model_1.predict("What are the metrics on which deepseek was evaluated")
print(response.text)

In [ ]:
base_model = TextGenerationModel.from_pretrained("text-bison@002")
response = base_model.predict("hat are the metrics on which deepseek was evaluated")
print(response.text)